In [ ]:
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense, Flatten
from tensorflow.keras import activations
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
def load_images_from_folder(folder, shape):
  images = []
  for filename in os.listdir(folder):
    img = Image.open(folder + '/' + filename)
    if img is not None:
      img = img.resize(shape)
      img = np.array(img)
      images.append(img)

  images = np.array(images)
  return images

In [ ]:
shape_of_img = (128, 128)
path1 = '/content/drive/MyDrive/NNFL Assignment - 2/Datasets/Images/class1_images'
im1 = load_images_from_folder(path1, shape_of_img)

path2 = '/content/drive/MyDrive/NNFL Assignment - 2/Datasets/Images/class2_images'
im2 = load_images_from_folder(path2, shape_of_img)

path3 = '/content/drive/MyDrive/NNFL Assignment - 2/Datasets/Images/class3_images'
im3 = load_images_from_folder(path3, shape_of_img)

In [ ]:
sz1 = im1.shape[0]
sz2 = im2.shape[0]
sz3 = im3.shape[0]


for i in range(sz1):
  im1[i][0][0][0] = 0.0

for i in range(sz2):
  im2[i][0][0][0] = 1.0

for i in range(sz3):
  im3[i][0][0][0] = 2.0

X = np.concatenate([im1, im2, im3], axis=0)
np.random.shuffle(X)

In [ ]:
Y = []

sz = X.shape[0]

for i in range(sz):
  Y.append(X[i][0][0][0])
  X[i][0][0][0] = 255.0

In [ ]:
Y = np.array(Y).reshape(len(Y), 1)
Y = to_categorical(Y, num_classes=3)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 8, kernel_size = 3, input_shape=(128, 128, 3), padding = "same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D(filters = 16, kernel_size = 3, padding = "same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D(filters = 32, kernel_size = 3, padding = "same"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'))
model.add(Activation('relu'))


model.add(Conv2D(filters = 64, kernel_size = 3, padding = "same"))
model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))


model.add(Dense(64))
model.add(Dropout(0.3))
model.add(Activation('relu'))


model.add(Dense(32))
model.add(Activation('relu'))


model.add(Dense(16))
model.add(Dropout(0.3))
model.add(Activation('relu'))


model.add(Dense(3))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 128, 128, 8)       224       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
activation_40 (Activation)   (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
activation_41 (Activation)   (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 128, 128, 32)     

In [ ]:
model.compile(optimizer = 'RMSprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
k_for_kfold = 10
size = int(X.shape[0] / k_for_kfold)
start = 0
end = size

histories = []

for kf in range(k_for_kfold):
  
  x = np.concatenate((X[:start, :], X[end:, :]), axis = 0)
  y = np.concatenate((Y[:start, :], Y[end:, :]), axis=0)
  tx = X[start:end, :]
  ty = Y[start:end, :]
  

  print("Round: " + str(kf+1) + ".....", end = " ")
  hist = model.fit(x, y, epochs = 50, validation_data=(tx, ty), verbose = 0)
  histories.append(hist)
  print("Done!")

  start += size
  end += size

Round: 1..... Done!
Round: 2..... Done!
Round: 3..... Done!
Round: 4..... Done!
Round: 5..... Done!
Round: 6..... Done!
Round: 7..... Done!
Round: 8..... Done!
Round: 9..... Done!
Round: 10..... Done!


In [ ]:
def one_hot_to_int(y):
  ret = []
  n, m = y.shape
  for i in range(n):
    for j in range(m):
      if (y[i][j] == 1.0):
        ret.append(j+1)
  
  return ret

In [ ]:
y_pred = model.predict(X)
y_pred = one_hot_to_int(y_pred)
y = one_hot_to_int(Y)

l = len(y)
correct, wrong = 0, 0

# overall accuracy
for i in range(l):
  if (y[i] == y_pred[i]):
    correct += 1
  else:
    wrong += 1

overall_accuracy = correct / (correct + wrong)
print("Overall accuracy (in %): " + str(overall_accuracy*100))

Overall accuracy (in %): 100.0


In [ ]:
# classwise accuracies
classwise_acc = []

for i in range(3):
  correct, wrong = 0, 0
  for j in range(l):
    if (y[j] == i+1):
      if (y_pred[j] == y[j]):
        correct += 1
      else:
        wrong += 1

  acc = correct / (correct + wrong)
  classwise_acc.append(acc)

print("Classwise accuracies:")
print("Class 1 accuracy (in %): " + str(classwise_acc[0]*100))
print("Class 2 accuracy (in %): " + str(classwise_acc[1]*100))
print("Class 3 accuracy (in %): " + str(classwise_acc[2]*100))

Classwise accuracies:
Class 1 accuracy (in %): 100.0
Class 2 accuracy (in %): 100.0
Class 3 accuracy (in %): 100.0
